# SageMaker processing を利用して前処理を行う(処理時間10分弱程度)

## 処理概要

1. 前処理用コンテナの作成(詳細は[こちら](./container/Dockerfile))
2. 前処理(詳細は[こちら](./container/preprocess_code/preprocess.py))をキック
    1. S3 から前処理用コンテナに先程作成した zip ファイルをダウンロード(自動)
    2. zip ファイルを解凍
    3. 画像を開いてヒストグラム平坦化
    4. numpy array に格納
    5. npy ファイルを出力
    6. 前処理用コンテナから S3 にデータをアップロード(自動)
3. 前処理した結果を確認

![](media/1_preprocess.png)

In [ ]:
import sagemaker
print(f'Current sagemaker Version ={sagemaker.__version__}')

注） このノートブックでは SageMaker SDK が 2.19.0 以上で動作します。上記の出力結果がそれ以前のバージョンになった際は、下記のセルの#を削除（コメントアウトを解除）して実行、Jupyterカーネルを再起動し、再度上記のセルを実行し、バージョンがアップデートされたことを確認してください。カーネルが再起動されない場合は、SageMaker SDK バージョン更新が反映されません。

In [ ]:
# !pip install -U --quiet "sagemaker>=2.19.0"

## 使用するライブラリ等の読み込みと設定ファイルの読み込み
自前のコンテナを SageMaker Processer で使うのと、その結果確認などに使うライブラリを事前に読み込む。また、[0_data_preparation.ipynb](./0_data_preparation.ipynb)の処理結果の格納場所などを引き継ぐため [setting.yaml](./setting.yaml) も読み込む

In [ ]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role
from sagemaker.s3 import parse_s3_url
import yaml,boto3, io
import numpy as np
from matplotlib import pyplot as plt
role = get_execution_role()
with open('./setting.yaml', 'r') as yml:
    config = yaml.load(yml)
name = config['name']
zip_dataset_s3_uri = config['zip_dataset_s3_uri']
timestamp = config['timestamp']
print(f'role: {role}')
print(f'name: {name}')
print(f'zip_dataset_s3_uri: {zip_dataset_s3_uri}')
print(f'timestamp: {timestamp}')

## 前処理用コンテナの作成
* SageMaker では組み込みコンテナとして [Apache Spark](https://docs.aws.amazon.com/sagemaker/latest/dg/use-spark-processing-container.html) と [scikit-learn](https://docs.aws.amazon.com/sagemaker/latest/dg/use-scikit-learn-processing-container.html) があるが、画像処理を扱うためのコンテナがない(scikit-image, opencv, pillow など)
* bring your own container することは可能なので、イメージを[作成](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-container-run-scripts.html)する
* SageMaker Notebook では Docker がプリインストールされているのでそのまま利用する
    1. ローカルでイメージをビルドする
    2. Elastic Container Registry の リポジトリにイメージを push する
    3. push した Image を利用して前処理を行う

In [ ]:
!cat ./container/Dockerfile

In [ ]:
# Image のビルド
%cd container
!docker build -t sagemaker-tf-handson-{name}-{timestamp} .
%cd ../

In [ ]:
import boto3

# boto3の機能を使ってリポジトリ名に必要な情報を取得する
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name
tag = ':latest'

ecr_repository = f'sagemaker-tf-handson-{name}-{timestamp}'
image_uri = f'{account_id}.dkr.ecr.{region}.amazonaws.com/{ecr_repository+tag}'

!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
 
# リポジトリの作成
# すでにある場合はこのコマンドは必要ない
!aws ecr create-repository --repository-name $ecr_repository
 
!docker build -t {ecr_repository} .
!docker tag {ecr_repository + tag} $image_uri
!docker push $image_uri

print(f'コンテナは {image_uri} へ登録されています。')

## Processor インスタンス作成
前処理を行うためのプロセッサーインスタンスを作成して、ジョブを開始する。 processor が処理を行う際のディレクトリとジョブ名を予め設定する（ジョブをキックするときに引数で指定する）

In [ ]:
processing_input_dir = '/opt/ml/processing/input'
processing_output_train_dir = '/opt/ml/processing/train'
processing_output_test_dir = '/opt/ml/processing/test'
job_name = f'sagemaker-preprocess-handson-{name}'

## 前処理を開始する
前処理を行うコードを確認する。前処理を行うコードは `./preprocess_script/preprocess.py` に格納

In [ ]:
!pygmentize ./preprocess_script/preprocess.py

自前の Docker イメージで処理するための ScriptProcessor クラスから processor インスタンスを生成する。 `image_uri ` に先程 `push` したイメージの URI を指定する。また、今回作成した イメージは `python3` で Python 3.7 へパスが通っているため、command に `python3` を指定する。
詳細は [ScriptProcessor]() を参照

In [ ]:
processor = ScriptProcessor(base_job_name=job_name,
                                   image_uri=image_uri,
                                   command=['python3'],
                                   role=role,
                                   instance_count=1,
                                   instance_type='ml.c5.xlarge'
                                  )

processor インスタンスを生成したら、 `run` メソッドで SageMaker Processing を走らせる。 `code` 引数に処理が書かれている .py ファイルを指定するが、S3 のパスでも良い。複数人で開発する際には、独自のスクリプトが乱立しやすいので、 S3 のパスを指定するか、 github や codecommit などのリポジトリから pull して使うか、Docker イメージに内包させて、code 引数で指定する.py ファイルには内包したスクリプトをキックさせるだけにする、など運用を考慮するとよい。arguments 引数で処理スクリプトに値を引き継ぎできる。

詳細は[こちら](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html?highlight=ScriptProcessor#sagemaker.processing.ScriptProcessor.run)

In [ ]:
# zip ファイルを解凍してヒストグラム平坦化してnpyファイルにまとめる前処理を実行
processor.run(code='./preprocess_script/preprocess.py', # S3 の URI でも可
                     inputs=[ProcessingInput(source=zip_dataset_s3_uri,destination=processing_input_dir)],
                     outputs=[
                         ProcessingOutput(output_name='train',source=processing_output_train_dir),
                         ProcessingOutput(output_name='test',source=processing_output_test_dir)],
                      arguments=[
                          '--hist-flatten', 'True',
                          '--input-dir',processing_input_dir,
                          '--output-train-dir',processing_output_train_dir,
                          '--output-test-dir',processing_output_test_dir
                      ]
                    )

describe メソッドを利用することでジョブの実行結果（データの出力先など）がわかる

In [ ]:
# ジョブの詳細を確認(前処理結果の格納先がわかる)
processor_description = processor.jobs[-1].describe()
print(processor_description)

## 実行結果確認
S3 に処理結果が格納されている。今回は npy ファイルを出力する処理のため、numpy でロードできるか、またその結果を目視で確認する

In [ ]:
train_data_uri = processor_description['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri']
test_data_uri = processor_description['ProcessingOutputConfig']['Outputs'][1]['S3Output']['S3Uri']
print(f'train_data_uri: {train_data_uri}')
print(f'test_data_uri: {test_data_uri}')

In [ ]:
bucket,train_key = parse_s3_url(train_data_uri)
bucket,test_key = parse_s3_url(test_data_uri)
s3 = boto3.client('s3')
obj_list=s3.list_objects_v2(Bucket=bucket, Prefix=train_key)
file=[]
for contents in obj_list['Contents']:
    file.append(contents['Key'])
obj_list=s3.list_objects_v2(Bucket=bucket, Prefix=test_key)
for contents in obj_list['Contents']:
    file.append(contents['Key'])

print(file)

In [ ]:
# train_x で100 番目のデータのみを確認
res = boto3.client('s3').get_object(Bucket = bucket, Key = file[0])["Body"].read()
train_x = np.load(io.BytesIO(res))
plt.imshow(train_x[100,:,:,0],'gray')